In [ ]:
import json
import os
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QLabel, QPushButton
from PyQt5.QtGui import QPixmap, QPainter
from PyQt5.QtCore import Qt
import pyqtgraph as pg


class ProgenWorldVisualizer(QMainWindow):
    def __init__(self, map_path, building_def_path):
        super().__init__()
        self.map_path = map_path
        self.building_def_path = building_def_path
        self.setWindowTitle('Progen World Map Viewer')

        # Load main map
        with open(map_path, 'r') as f:
            self.data = json.load(f)
        self.nodes = self.data.get('nodes', [])

        # Load building definitions
        with open(building_def_path, 'r') as f:
            self.building_defs = self._load_building_defs(json.load(f))

        # GUI layout
        self.main_widget = QWidget()
        self.setCentralWidget(self.main_widget)
        layout = QVBoxLayout(self.main_widget)

        self.title_label = QLabel("Progen Map Viewer")
        self.title_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.title_label)

        self.plot_widget = pg.PlotWidget()
        layout.addWidget(self.plot_widget)

        self.save_button = QPushButton('Save Image')
        self.save_button.clicked.connect(self.save)
        layout.addWidget(self.save_button)

        self.plot_widget.setBackground('#F0F8FF')
        self.plot_widget.showGrid(True, True, alpha=0.3)
        self.plot_widget.setAspectLocked(True)
        self.plot_widget.setMouseEnabled(x=True, y=True)
        self.plot_widget.setMenuEnabled(False)

        self.resize(1280, 720)
        self.draw_map()

    def _load_building_defs(self, defs_json):
        result = {}
        for b in defs_json.get('buildings', []):
            btype = b.get('type')
            bounds = b.get('bounds', {})
            if btype and 'width' in bounds and 'height' in bounds:
                result[btype] = {
                    'width': bounds['width'],
                    'height': bounds['height'],
                    'rotation': bounds.get('rotation', 0)
                }
        return result

    def draw_map(self):
        self.plot_widget.clear()

        building_count = 0
        road_count = 0
        tree_count = 0

        for idx, node in enumerate(self.nodes):
            instance_name = node.get('instance_name', '')
            props = node.get('properties', {})
            loc = props.get('location', {})
            ori = props.get('orientation', {})

            x = loc.get('x', 0)
            y = loc.get('y', 0)
            yaw = ori.get('yaw', 0)

            print(f"[{idx}] Processing node: {instance_name}")

            # Buildings
            if instance_name.startswith('BP_Building'):
                btype = instance_name
                size = self.building_defs.get(btype, {'width': 50, 'height': 40})
                width = size['width'] * 50
                height = size['height'] * 50

                rect = pg.QtWidgets.QGraphicsRectItem(x, y, width, height)
                rect.setTransformOriginPoint(x + width / 2, y + height / 2)
                rect.setRotation(yaw)
                rect.setPen(pg.mkPen('#2C3E50', width=1))
                rect.setBrush(pg.mkBrush('#34495E'))
                self.plot_widget.addItem(rect)
                building_count += 1

            # # Roads
            # elif instance_name.startswith('BP_Road'):
            #     seg = node.get('segment_assignment', {})
            #     start = seg.get('start', {})
            #     end = seg.get('end', {})
            #     if start and end:
            #         x1, y1 = start.get('x', 0), start.get('y', 0)
            #         x2, y2 = end.get('x', 0), end.get('y', 0)
            #         line = pg.PlotDataItem(
            #             x=[x1, x2],
            #             y=[y1, y2],
            #             pen=pg.mkPen('#1E3F66', width=2.5),
            #             antialias=True
            #         )
            #         self.plot_widget.addItem(line)
            #         road_count += 1

            # # Trees
            # elif instance_name.startswith('BP_Tree'):
            #     radius = 500  # 以实际地图单位为单位的大小（可调）
            #     circle = pg.QtWidgets.QGraphicsEllipseItem(x - radius, y - radius, radius * 2, radius * 2)
            #     circle.setPen(pg.mkPen('w'))
            #     circle.setBrush(pg.mkBrush('#27AE60'))
            #     self.plot_widget.addItem(circle)
            #     tree_count += 1

        print(f"✅ Drawing complete: {building_count} buildings, {road_count} roads, {tree_count} trees.")

    def save(self):
        out_path = os.path.join(os.path.dirname(self.map_path), 'progen_map_output.png')
        pixmap = QPixmap(self.plot_widget.size())
        painter = QPainter(pixmap)
        self.plot_widget.render(painter)
        painter.end()
        if pixmap.save(out_path, 'PNG'):
            print(f'Image saved at {out_path}')
        else:
            print('❌ Failed to save image')


def main():
    import sys
    app = QApplication(sys.argv)

    # TODO: 修改为你本地的两个文件路径 👇
    map_path = "D:\\LLMDelivery-LJ\\Test_Data\\test\\progen_world.json"
    building_def_path = "D:\\LLMDelivery-LJ\\Test_Data\\test\\buildings.json"

    viewer = ProgenWorldVisualizer(map_path, building_def_path)
    viewer.show()
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()


[0] Processing node: BP_Road1_C
[1] Processing node: BP_Road1_C
[2] Processing node: BP_Road1_C
[3] Processing node: BP_Road1_C
[4] Processing node: BP_Road1_C
[5] Processing node: BP_Road1_C
[6] Processing node: BP_Road1_C
[7] Processing node: BP_Road1_C
[8] Processing node: BP_Road1_C
[9] Processing node: BP_Road1_C
[10] Processing node: BP_Road1_C
[11] Processing node: BP_Road1_C
[12] Processing node: BP_Road1_C
[13] Processing node: BP_Road1_C
[14] Processing node: BP_Road1_C
[15] Processing node: BP_Building_12_C
[16] Processing node: BP_Building_09_C
[17] Processing node: BP_Building_16_C
[18] Processing node: BP_Building_05_C
[19] Processing node: BP_Building_75_C
[20] Processing node: BP_Building_75_C
[21] Processing node: BP_Building_76_C
[22] Processing node: BP_Building_75_C
[23] Processing node: BP_Building_75_C
[24] Processing node: BP_Building_31_C
[25] Processing node: BP_Building_75_C
[26] Processing node: BP_Building_98_C
[27] Processing node: BP_Building_36_C
[28] Pro

SystemExit: 0

: 